<a href="https://colab.research.google.com/github/anaswara21/test/blob/main/2_Longformer_Embedding_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import nltk
import sklearn
import pandas as pd
from tqdm.notebook import tqdm, trange
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
import tensorflow_hub as hub
import tensorflow as tf

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/context_and_emotion/dataset

/content/drive/MyDrive/context_and_emotion/dataset


In [ ]:
dfTrain = pd.read_csv("Dataset_PreProcessed.csv")

In [ ]:
embeddings=[]

In [ ]:
import torch

from transformers import LongformerModel, AutoTokenizer

model = LongformerModel.from_pretrained("allenai/longformer-base-4096")

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

for i in trange(len(dfTrain['Text_of_Speech'])):

    input_ids = torch.tensor(tokenizer.encode(dfTrain['Text_of_Speech'][i])).unsqueeze(0)  # batch of size 1

    attention_mask = torch.ones(

        input_ids.shape, dtype=torch.long, device=input_ids.device

    )  # initialize to local attention

    global_attention_mask = torch.zeros(

        input_ids.shape, dtype=torch.long, device=input_ids.device

    )  # initialize to global attention to be deactivated for all tokens

    global_attention_mask[

        :,

        [

            0,
            -1

        ],

    ] = 1  # Set global attention to random tokens for the sake of this example

    # Usually, set global attention based on the task. For example,

    # classification: the <s> token

    # QA: question tokens

    # LM: potentially on the beginning of sentences and paragraphs

    outputs = model(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)

    sequence_output = outputs.last_hidden_state

    pooled_output = outputs.pooler_output
    x=pooled_output.detach().numpy()
    embeddings.append(x)



  0%|          | 0/2010 [00:00<?, ?it/s]

In [ ]:
len(embeddings[0][0])

768

In [ ]:
twoDimentionalEmbedding =[]

In [ ]:
for array in trange(len(embeddings)):
    twoDimentionalEmbedding.append(embeddings[array][0])

  0%|          | 0/2010 [00:00<?, ?it/s]

In [ ]:
len(twoDimentionalEmbedding[0])

768

In [ ]:
columnNames=[]
for name in range(768):
    columnNames.append(name)

In [ ]:
dfTrainEmbedded = pd.DataFrame(twoDimentionalEmbedding, columns =columnNames, dtype = float)

In [ ]:
dfTrainEmbedded['Emotion'] = dfTrain['Context']

In [ ]:
dfTrainEmbedded['Context'] = dfTrain['Context']

In [ ]:
dfTrainEmbedded.to_csv('Dataset_Longformer_Embedded.csv', index=False)